In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

from matplotlib.colors import BoundaryNorm, ListedColormap

import lysis

In [2]:
dataset_type = np.dtype([("exp_code", np.str_, 15), ("file_code", np.str_, 40), ("descriptor", np.str_, 40)])
code_type = np.dtype([("file_code", np.str_, 40), ("descriptor", np.str_, 40)])

In [3]:
datasets = np.array([("2023-01-24-0200", "", "Physiological Kd"),
                     ("2023-01-24-0000", "_Kd0236", "10x bigger"),
                     ("2023-01-24-0100", "_Kd00020036", "10x smaller"),], 
                    dtype=dataset_type)
programs = np.array([("_always", "Always bind"),
                     ("_along", "Diffuse along clot"),
                     ("_into", "Diffuse into clot"),
                     ("_into_and_along", "Diffuse into and along clot - BUGGED"),
                     ("_into_and_along_fixed", "Diffuse into and along clot - FIXED")
                    ],
                    dtype=code_type)

In [4]:
index = pd.MultiIndex.from_product([datasets["descriptor"], programs["descriptor"]], names=["data", "program"])
statistics = ['Mean front velocity', 'Mean of front velocity Standard Deviation']
results = pd.DataFrame(index=index, columns=statistics)
front_velocity_table = pd.DataFrame(index=programs['descriptor'], columns=datasets['descriptor'])

In [5]:
def load_files(exp, file_code):
    deg = np.fromfile(os.path.join(e.os_path, "deg" + file_code))
    tsave = np.fromfile(os.path.join(e.os_path, "tsave" + file_code))
    mfpt = np.fromfile(os.path.join(e.os_path, "mfpt" + file_code))
    deg = deg.reshape(exp.macro_params.total_trials, exp.macro_params.number_of_saves, exp.macro_params.total_edges)
    tsave = tsave.reshape(exp.macro_params.total_trials, exp.macro_params.number_of_saves)
    return deg, tsave, mfpt

def map_fortran_deg(deg):
    mapped_deg = -deg
    mapped_deg[deg == 0] = float('inf')
    mapped_deg[deg == -1] = 0
    return mapped_deg

def calculate_time_row_exposed(exp, deg):
    exposed_time = np.empty((exp.macro_params.total_trials, exp.macro_params.rows-1, exp.macro_params.cols), 
                        dtype=np.float_)
    for run in range(exp.macro_params.total_trials):
        for j in range(exp.macro_params.cols):
            for i in range(exp.macro_params.rows-1):
                if i == 0:
                    exposed_time[run, i, j] = 0;
                else:
                    k = lysis.to_fortran_edge_index(i, j, exp.macro_params.rows, exp.macro_params.cols)
                    exposed_time[run, i, j] = max(exposed_time[run, i-1, j], deg[run, exp.macro_params.number_of_saves-1, k])
    #exposed_time = 10* np.ceil(exposed_time / 10)
    return exposed_time / 60

def find_degradation_fronts(exp, exposed_time, y_distance):
    deg_fronts = []
    for r in range(exp.macro_params.total_trials):
        run_deg_fronts = []
        for j in range(exp.macro_params.cols):
            col_deg_front = []
            for i in range(1, exp.macro_params.rows-1):
                if exposed_time[r, i-1, j] < exposed_time[r, i, j] < float('inf'):
                    col_deg_front.append([exposed_time[r, i, j], y_distance[i]])
            run_deg_fronts.append(np.array(col_deg_front).T)
        deg_fronts.append(run_deg_fronts)
    return deg_fronts

def mean_front_velocity(exp, deg_fronts):
    run_mean_velocity = np.empty(exp.macro_params.total_trials, dtype=np.float_)
    run_std_velocity = np.empty(exp.macro_params.total_trials, dtype=np.float_)
    for run in range(exp.macro_params.total_trials):
        front_velocity = np.empty(exp.macro_params.cols, dtype=np.float_)
        for j in range(exp.macro_params.cols):
            b, m = np.polynomial.polynomial.polyfit(deg_fronts[run][j][0], deg_fronts[run][j][1], 1)
            front_velocity[j] = m
        run_mean_velocity[run] = np.mean(front_velocity)
        run_std_velocity[run] = np.std(front_velocity)
    return np.mean(run_mean_velocity), np.mean(run_std_velocity)

def plot_front_degradation(exp, file_code, deg_fronts):
    plt.ioff()
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_axis_on()
    ax.set_xlim(0, exp.macro_params.total_time / 60)
    ax.set_ylim((exp.macro_params.empty_rows-1)*e.macro_params.grid_node_distance,
               (exp.macro_params.rows-1)*exp.macro_params.grid_node_distance)
    for run in range(exp.macro_params.total_trials):
        for j in range(exp.macro_params.cols):
            plt.plot(deg_fronts[run][j][0], deg_fronts[run][j][1], linewidth=1)
    fig.savefig(os.path.join(exp.os_path, "deg_fronts" + file_code[:-4] + ".png"))
    plt.close()

In [6]:
for data in datasets:
    e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code=data["exp_code"])
    e.read_file()
    y_distance = np.arange(e.macro_params.rows-1)*e.macro_params.grid_node_distance 
    for program in programs:
        file_code = f"_PLG2_tPA01{data['file_code']}{program['file_code']}_Q2.dat"
        print(file_code)
        deg, tsave, mfpt = load_files(e, file_code)
        deg = map_fortran_deg(deg)
        exposed_time = calculate_time_row_exposed(e, deg)
        deg_fronts = find_degradation_fronts(e, exposed_time, y_distance)
        plot_front_degradation(e, file_code, deg_fronts)
        m, sd = mean_front_velocity(e, deg_fronts)
        results.loc[(data['descriptor'], program['descriptor']), 'Mean front velocity'] = m
        results.loc[(data['descriptor'], program['descriptor']), 'Mean of front velocity Standard Deviation'] = sd
        front_velocity_table.loc[program['descriptor'], data['descriptor']] = f"{m:.2f} " + u"\u00B1" + f" {sd:.2f}"

_PLG2_tPA01_always_Q2.dat
_PLG2_tPA01_along_Q2.dat
_PLG2_tPA01_into_Q2.dat
_PLG2_tPA01_into_and_along_Q2.dat
_PLG2_tPA01_into_and_along_fixed_Q2.dat
_PLG2_tPA01_Kd0236_always_Q2.dat
_PLG2_tPA01_Kd0236_along_Q2.dat
_PLG2_tPA01_Kd0236_into_Q2.dat
_PLG2_tPA01_Kd0236_into_and_along_Q2.dat
_PLG2_tPA01_Kd0236_into_and_along_fixed_Q2.dat
_PLG2_tPA01_Kd00020036_always_Q2.dat
_PLG2_tPA01_Kd00020036_along_Q2.dat
_PLG2_tPA01_Kd00020036_into_Q2.dat
_PLG2_tPA01_Kd00020036_into_and_along_Q2.dat
_PLG2_tPA01_Kd00020036_into_and_along_fixed_Q2.dat


In [7]:
front_velocity_table

,Physiological Kd,10x bigger,10x smaller
Always bind,6.80 ± 0.05,10.64 ± 0.16,6.25 ± 0.05
Diffuse along clot,4.48 ± 0.06,9.47 ± 0.15,1.44 ± 0.08
Diffuse into clot,10.84 ± 1.77,15.45 ± 0.60,2.06 ± 0.60
Diffuse into and along clot - BUGGED,11.00 ± 1.77,15.40 ± 0.62,2.14 ± 0.64
Diffuse into and along clot - FIXED,6.20 ± 0.27,10.10 ± 0.18,1.42 ± 0.09


In [8]:
results

Mean front velocity  \
data             program                                                    
Physiological Kd Always bind                                     6.800348   
                 Diffuse along clot                              4.475497   
                 Diffuse into clot                              10.840919   
                 Diffuse into and along clot - BUGGED           10.995715   
                 Diffuse into and along clot - FIXED             6.196575   
10x bigger       Always bind                                    10.636808   
                 Diffuse along clot                              9.473842   
                 Diffuse into clot                              15.452165   
                 Diffuse into and along clot - BUGGED           15.400921   
                 Diffuse into and along clot - FIXED              10.0976   
10x smaller      Always bind                                     6.248852   
                 Diffuse along clot                              1.436927   
                 Diffuse into clot                               2.062396   
                 Diffuse into and along clot - BUGGED            2.138098   
                 Diffuse into and along clot - FIXED             1.415761   

                                                      Mean of front velocity Standard Deviation  
data             program                                                                         
Physiological Kd Always bind                                                            0.04797  
                 Diffuse along clot                                                    0.057457  
                 Diffuse into clot                                                      1.76744  
                 Diffuse into and along clot - BUGGED                                  1.767496  
                 Diffuse into and along clot - FIXED                                    0.26612  
10x bigger       Always bind                                                           0.155186  
                 Diffuse along clot                                                    0.152027  
                 Diffuse into clot                                                     0.596008  
                 Diffuse into and along clot - BUGGED                                  0.616205  
                 Diffuse into and along clot - FIXED                                   0.177387  
10x smaller      Always bind                                                           0.045949  
                 Diffuse along clot                                                     0.07847  
                 Diffuse into clot                                                     0.597701  
                 Diffuse into and along clot - BUGGED                                  0.636982  
                 Diffuse into and along clot - FIXED                                   0.088812

In [9]:
type((None,))

tuple